In [201]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.metrics import classification_report

In [202]:
df = pd.read_csv('../data/waterQuality1.csv')
# https://www.kaggle.com/datasets/mssmartypants/water-quality/data

In [203]:
df = df.drop(df[df['ammonia'] == "#NUM!"].index)
df['ammonia'] = df['ammonia'].astype('float')
df['is_safe'] = df['is_safe'].astype('float')

In [204]:
# df.info()

In [205]:

x = np.array(df.drop(['is_safe'], axis=1))
y = np.array(df['is_safe'].copy())

In [206]:
from model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y, testsize=0.2, debug=True)
x_train


Data  Size: 7996
Train Size: 6397
Test  Size: 1599
Train Size + Test Size: 7996


array([[4.000e-02, 2.546e+01, 1.000e-02, ..., 6.000e-02, 1.900e-01,
        6.000e-02],
       [2.290e+00, 5.420e+00, 1.800e-01, ..., 5.000e-02, 1.000e-01,
        8.000e-02],
       [1.000e-02, 2.551e+01, 4.000e-02, ..., 2.000e-02, 1.000e-01,
        4.000e-02],
       ...,
       [8.000e-02, 9.880e+00, 3.000e-02, ..., 9.000e-02, 1.000e-02,
        7.000e-02],
       [3.100e-01, 1.238e+01, 3.900e-01, ..., 8.000e-02, 3.200e-01,
        0.000e+00],
       [2.700e+00, 1.413e+01, 4.000e-02, ..., 6.000e-02, 5.000e-02,
        2.000e-02]])

In [207]:
class SupportVectorMachine:
    def __init__(self, C=10, kernel='linear', learning_rate=0.001, lambda_param=0.01, n_iters=1000, debug=False) -> None:
        self.c = C
        self.kernel = kernel

        self.w = None
        self.b = None
        self.lerning_rate = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters

        if debug:
            print("C: {}\nLearning Rate: {}\nLambda Param: {}\nN_Iters: {}".format(self.c, self.lerning_rate, self.lambda_param, self.n_iters))

    def score(self, x_test, y_test):
        size = x_test.shape[0]      # Get data size
        predict_correct = 0         # Count TP or TF

        for index in range(size):

            predict = self.predict(x_test[index].reshape(1,-1)) # Predict
            # print(predict, y_test[index])

            if (predict == y_test[index]):          # If correct predict
                predict_correct += 1
        
        print("Accuracy: ", (predict_correct / size) * 100.00)     


    def higeLoss(self,w, x, y, b):
        fx = y * (np.dot(w, x) + b)

        if fx >= 1:
            return 0
        elif fx < 1:
            return 1 - fx
        

    def fit(self, x, y)-> None:
        sample_size, feature_size = x.shape
        
        self.w = np.zeros(feature_size) # init feature
        self.b = 0                      # init bias

        for _ in range(self.n_iters):
            for n_sameple, x_sample in enumerate(x):
                # constrain = y[n_sameple] * (np.dot(self.w, x_sample + self.b))
                constrain  = np.dot(self.w, self.w) + (self.c * self.higeLoss(self.w ,x_sample, y[n_sameple],self.b))
                
                if constrain < 1:
                    self.w = self.w - (self.lerning_rate * (self.lambda_param * self.w - (y[n_sameple] * x_sample)) )
                else:
                    self.w = self.w - (self.lerning_rate * self.lambda_param * self.w)
                



    def predict(self , x):
        approx = np.dot(x, self.w) - self.b
        if approx > 1:
            return 1
        else:
            return 0

In [208]:
model1 = SupportVectorMachine(C=100, debug=True)
model1.fit(x_train, y_train)
model1.score(x_test, y_test)

C: 100
Learning Rate: 0.001
Lambda Param: 0.01
N_Iters: 1000
Accuracy:  88.49280800500313


In [209]:
# model = svm.SVC(kernel='linear')
# model.fit(x_train, y_train)

# y_pred = model.predict(x_test)
# model.score(x_test, y_test)


# print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
# # print("Precision:",metrics.precision_score(y_test, y_pred, average="weighted"))
# # print("Recall:",metrics.f1_score(y_test, y_pred, average="macro"))
# print(classification_report(y_test, y_pred))
